In [95]:
import pandas as pd
import numpy as np
import time

In [115]:
financial_securitie = {
    "id":0,
    "timestamp":1581717368.347977,
    "Synthetic_index":100,
    "owner":"Alex Méndez",
    "entity":"banco santander",
    "underlying_securities":[
        {
            "id":0,
            "type":"ACCION",
            "price":70,
            "weight":5
        },
        { 
            "id":1,
            "type":"DEUDA",
            "price":100,
            "weight":15
        }
        ]
}
peticion = {
    "VF":"0",
    "idAS":1,
    "precio":90

}

NameError: name 'ISODate' is not defined

In [97]:
def calculate_synthetic_index(precio_peticion,precio_antiguo,peso,indice_sintetico_inicial):
    """
    Funcion que recibe 4 parametros para calcular el indice sintetico al actualizar el precio:
    - precio_peticion: precio al que se actualiza
    - precio_antiguo: precio antes de la actualizacion
    - peso de la accion
    - indice sintetico inical
    """
    Rit = (precio_peticion/precio_antiguo)-1
    Rt = peso*Rit
    isf = indice_sintetico_inicial*(1-Rt)
    return isf

In [98]:
def update_financial_securitie(financial_securitie,peticion):    
    activos = financial_securitie["activos_subyacentes"]
    isi = financial_securitie["Synthetic_index"]
    p_peticion = peticion["price"]
    for i in range(len(activos)):
        if activos[i]["id"]== peticion["idAS"]:
            peso = activos[i]["weight"]
            indice_sintetico_final = calculate_synthetic_index(p_peticion,activos[i]["precio"],peso,isi)
            activos[i]['price'] = p_peticion
            fs_final = {
                        "id":financial_securitie["id"]+1,
                        "timestamp":time.time(),
                        "Synthetic_index":indice_sintetico_final,
                        "owner":"pepe",
                        "entity":"banco santander",
                        "underlying_securities":activos
                        }
            return fs_final
fs_final = update_financial_securitie(financial_securitie,peticion)
fs_final

{'id': 1,
 'timestamp': 1581840620.3934588,
 'Synthetic_index': 249.99999999999994,
 'dueño': 'pepe',
 'entidad': 'banco santander',
 'activos_subyacentes': [{'id': 0, 'type': 'ACCION', 'precio': 70, 'peso': 5},
  {'id': 1, 'type': 'DEUDA', 'precio': 90, 'peso': 15}]}

In [116]:
peticiones = {
    0:{
        "id_fs":0,
        "idAS":0,
        "price":90  
    }
}
peticiones[0]["id_fs"]

0

In [100]:
def calculate_Rt(activos,peticiones):
    Rt = 0
    for ind in range(len(peticiones)):
        for i in range(len(activos)):
            if activos[i]["id"]==peticiones[ind]["idAS"]:
                rit=(peticiones[ind]["price"]/activos[i]["price"])-1
                Rt+=activos[i]["weight"]*rit
                activos[i]["price"] = peticiones[ind]["price"]
        return Rt, activos

In [101]:
Rt,activos = calculate_Rt(financial_securitie["activos_subyacentes"],peticiones)
si = 100*(1+Rt)
    

In [102]:
def update_financial_securitie_2(financial_securitie,peticion):    
    activos = financial_securitie["underlying_securities"]
    isi = financial_securitie["Synthetic_index"]
    p_peticion = peticion["price"]
    
    Rt,activos = calculate_Rt(financial_securitie["underlying_securities"],peticiones)
    si = isi*(1+Rt)
    
    fs_final = {
                "id":financial_securitie["id"]+1,
                "timestamp":time.time(),
                "Synthetic_index":si,
                "owner":"pepe",
                "entity":"banco santander",
                "underlying_securities":activos
                }
    return fs_final
fs_final = update_financial_securitie_2(financial_securitie,peticion)
fs_final

{'id': 1,
 'timestamp': 1581840623.570646,
 'Synthetic_index': 100.0,
 'dueño': 'pepe',
 'entidad': 'banco santander',
 'activos_subyacentes': [{'id': 0, 'type': 'ACCION', 'precio': 90, 'peso': 5},
  {'id': 1, 'type': 'DEUDA', 'precio': 90, 'peso': 15}]}

In [103]:
from pymongo import MongoClient
import os
from dotenv import load_dotenv
load_dotenv()

#Get Password
connection = os.getenv('URLMONGO')
#Connect to DB
client = MongoClient(connection)
def connectCollection(database, collection):
    db = client[database]
    coll = db[collection]
    return db, coll

In [104]:
db,coll = connectCollection('ets_factory','financial_securities')

In [113]:
coll.insert(financial_securitie)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  """Entry point for launching an IPython kernel.


ObjectId('5e48fdeb8dbb46c2fb90ef75')

In [114]:
list(coll.find({}))

[{'_id': ObjectId('5e48f9028dbb46c2fb90ef73'),
  'id': 0,
  'timestamp': 1581717368.347977,
  'Synthetic_index': 100,
  'dueño': 'pepe',
  'entidad': 'banco santander',
  'activos_subyacentes': [{'id': 0, 'type': 'ACCION', 'precio': 90, 'peso': 5},
   {'id': 1, 'type': 'DEUDA', 'precio': 90, 'peso': 15}]},
 {'_id': ObjectId('5e48fdc78dbb46c2fb90ef74'),
  'id': 0,
  'timestamp': 1581717368.347977,
  'Synthetic_index': 100,
  'owner': 'pepe',
  'entity': 'banco santander',
  'underlying_securities': [{'id': 0,
    'type': 'ACCION',
    'price': 70,
    'weight': 5},
   {'id': 1, 'type': 'DEUDA', 'price': 100, 'weight': 15}]},
 {'_id': ObjectId('5e48fdeb8dbb46c2fb90ef75'),
  'id': 0,
  'timestamp': 1581717368.347977,
  'Synthetic_index': 100,
  'owner': 'Alex Méndez',
  'entity': 'banco santander',
  'underlying_securities': [{'id': 0,
    'type': 'ACCION',
    'price': 70,
    'weight': 5},
   {'id': 1, 'type': 'DEUDA', 'price': 100, 'weight': 15}]}]

In [117]:
financial_securitie = list(coll.find({'id':peticiones[0]["id_fs"]}))[-1]

In [118]:
financial_securitie

{'_id': ObjectId('5e48fdeb8dbb46c2fb90ef75'),
 'id': 0,
 'timestamp': 1581717368.347977,
 'Synthetic_index': 100,
 'owner': 'Alex Méndez',
 'entity': 'banco santander',
 'underlying_securities': [{'id': 0,
   'type': 'ACCION',
   'price': 70,
   'weight': 5},
  {'id': 1, 'type': 'DEUDA', 'price': 100, 'weight': 15}]}